In [ ]:
##part 1
##use models with default
##use data set with H/A +1, -1
##do full window for now

##next:
##check if just excluding first 10 days helps (chaotic)
##check if different windows help

##next
## can try tuning (for loops by hand, or ... use grid_search (use ML mastery code))
##-I think tuning will be faster ... just do by hand ... loop over the possible things 
##-ONE for loop over i = (a,b,c,d)... for each model i[0]

##Orrr can try adding features ... here we have to worry about:
##-adding basic features eg pp, and correct fo%
##-scaling numericals
##-dummy vars for categoricals (are there any?) besides H/A
##-num_windows and which lengths for moving avgs
##-filtering the features for increasing complexity inteligently
##-There is a dicotemy: 
##(a)use H/A + numerics or  ... here I think it can be made more like time-series
##(b) just use mumerics (moving avg) ... here I think the order of the games is not important (note Leung did this, and random train)



In [1]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

Pischsada features #variation of Ottawa

Goals for Total number of goals scored so far this season

Goals against

Total number of goals conceded so far thisseason

Goals Differential Goals for – Goals against

Power Play Success Rate Ratio – scoring a goal when 5 on 4

Power Kill Success Rate Ratio – not conceding a goal when 4 on 5

Shot % Goal scored/shots takenaaa

Save % Goals conceded/shots saved

Winning Streak Number of consecutive games won

Conference Standing Latest ranking on conference table

Fenwick Close % Possession ratio

PDO Luck parameter

5/5 Goal For/Against Ratio – 5 on 5 Goals For/Against

In [219]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models

##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()


In [221]:
y_Pis

,date,full_date,game_id,Open,goal_difference,won
0,119,20130119,2012020001,-105,-2.0,0
1,119,20130119,2012020002,-120,-3.0,0
2,119,20130119,2012020003,-130,-3.0,0
3,119,20130119,2012020004,-120,2.0,1
4,119,20130119,2012020005,-135,-1.0,0
...,...,...,...,...,...,...
715,427,20130427,2012020717,105,-3.0,0
716,427,20130427,2012020718,105,5.0,1
717,427,20130427,2012020719,-155,-2.0,0
718,427,20130427,2012020720,-150,1.0,1


In [287]:
y = y_Pis.loc[13:, 'won'].copy()
X = data_dummies_Pis.copy()
X.shape
X[300:]

,ANA,ARI,BOS,BUF,CAR,CBJ,CGY,CHI,COL,DAL,...,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg,last_10_games_win%,win%
300,0,0,0,0,1,0,0,0,0,0,...,-29.0,-0.169975,0.012132,-0.097217,-0.007861,-0.026208,-0.040217,-0.034069,-0.166667,-0.264286
301,0,0,-1,0,0,0,0,0,0,0,...,0.0,0.015826,0.033673,0.093840,-0.012851,0.010568,0.008980,-0.002283,0.600000,0.158730
302,0,0,0,1,0,0,0,0,0,0,...,15.0,0.058811,-0.029061,0.018375,-0.015232,0.006996,0.059560,-0.008235,-0.350000,0.064593
303,0,0,0,0,0,0,0,0,0,0,...,1.0,0.023115,-0.102183,0.024286,-0.003073,-0.004798,0.039460,-0.007871,0.200000,0.100000
304,0,0,0,0,0,0,-1,0,0,0,...,-17.0,-0.143168,0.051861,-0.005852,-0.002913,-0.028267,-0.010704,-0.031179,-0.100000,-0.181579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0,0,0,0,0,0,0,0,0,-1,...,-19.0,-0.035630,-0.007143,-0.042334,0.015628,-0.011179,-0.064266,0.004450,-0.400000,-0.021277
703,0,0,0,0,0,0,0,0,0,0,...,-25.0,-0.094540,0.043558,-0.022948,-0.000104,-0.004858,-0.065536,-0.004962,-0.100000,-0.170213
704,-1,1,0,0,0,0,0,0,0,0,...,27.0,0.091067,0.065907,-0.002668,0.023944,0.002161,-0.018613,0.026105,0.200000,0.212766
705,0,0,0,0,0,0,0,0,0,0,...,11.0,0.029841,-0.001672,-0.038180,0.021633,-0.019687,0.023549,0.001946,0.000000,0.021277


In [301]:

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC(kernel = 'rbf')

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=5, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    model.fit(X.iloc[:300 :], y[:300])
    y_pred = model.predict(X.iloc[300:, :])
    y_test = y[300:].copy()
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    print(model, acc, f1)

SVC() 0.5552825552825553 0.6916524701873934
RidgeClassifier() 0.5773955773955773 0.6518218623481782
RandomForestClassifier(max_depth=5, random_state=0) 0.5724815724815725 0.6836363636363636
GaussianNB() 0.5282555282555282 0.6016597510373444
LogisticRegression(random_state=0) 0.5798525798525799 0.6613861386138614
BaggingClassifier() 0.5135135135135135 0.5499999999999999
GradientBoostingClassifier() 0.5257985257985258 0.5936842105263157
[00:06:39] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constra

/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



##hyper_parameters from here 
##https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
##for xgboost from here 
##https://machinelearningmastery.com/extreme-gradient-boosting-ensemble-in-python/

#xgb

trees = [10, 50, 100, 500, 1000, 5000]  #100  #num of trees
max_depth = range(1,11)  ##3-5
rates = [0.0001, 0.001, 0.01, 0.1, 1.0]  #0.1
subsample in arange(0.1, 1.1, 0.1):  #0.4, 0.5  ##this is 0.1, 0.2 ... 1.0 # % of features to sample


#svc 
kernels in [‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’] #if you use poly, then adjust degree
C in [100, 10, 1.0, 0.1, 0.001]

#gb

learning_rate in [0.001, 0.01, 0.1]
n_estimators [10, 100, 1000]
subsample in [0.5, 0.7, 1.0]
max_depth in [3, 7, 9]


#rfc
max_features [1 to 20]  #key
max_features in [‘sqrt’, ‘log2’]
n_estimators in [10, 100, 1000]

#bc
n_estimators in [10, 100, 1000]

svm_dic = {'kernels':[‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’]}
lrc_dic = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
lgr_hp_dic = {'solver': [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’], 'penalty' : [‘none’, ‘l1’, ‘l2’, ‘elasticnet’],
'C' :[100, 10, 1.0, 0.1, 0.01]}

In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


import matplotlib.pyplot as plt
import seaborn as sns


##couple evaluation functions ##removed model_name as variable
def evaluate_binary_classification(y_test, y_pred, y_proba=None, graph = False):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    #try:
    if y_proba != None:
        rocauc_score = roc_auc_score(y_test, y_proba)
    else:
        rocauc_score = "no roc"
    #except: 
    #    pass     
    cm = confusion_matrix(y_test, y_pred)
    if graph == True:
        sns.heatmap(cm, annot=True)
        plt.tight_layout()
        plt.title(f'{model_name}', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.show()
    print("accuracy: ", accuracy)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1 score: ", f1)
    print("rocauc: ", rocauc_score)
    print(cm)
    #return accuracy, precision, recall, f1, rocauc_score

def evaluate_regression(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("mae", mae)
    print("mse", mse)
    print('r2', r2)
    
##display null values


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [4]:

data = pd.read_csv("/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_bet_stats_mp.csv")
data.drop(columns=[ 'Unnamed: 0'], inplace=True)

In [5]:
data['won'] = data['won'].apply(int)
data_playoffs = data.loc[data['playoffGame'] == 1, :].copy()  #set aside playoff games ... probably won't use them.
data=  data.loc[data['playoffGame'] == 0, :].copy() 

#sorted(data.columns)

In [6]:
##not needed! Dates are already in Chrono order, nice eh? 20161022 < 20161201 < 20170120

def get_dates(X, season):  #season 20162017 int; returns the list of all dates in chronological order from that season 20161004, 20161005, ...
    X = X.loc[X['season'] == season, :].copy()
    dates_1 = list(set(X.loc[(X['date'] >= 900) & (X['date'] <= 1231) , :]['full_date']))  #2016 part
    dates_2 = list(set(X.loc[(X['date'] >= 100) & (X['date'] <= 800) , :]['full_date']))  #2017 part
    dates = dates_1 + dates_2 #all dates in order
    return dates

#sorted(data.columns)

In [7]:


def make_HA_diff(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    df_model = df_home.sub(df_visitor) 
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)

##try later maye 
def make_HA_concat(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    
    
    df_model = df_home.sub(df_visitor) 
    
    
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)


In [8]:
##have not written yet

def top3_max_val_params(model, X, dates, drop_firs=False):
    pass
    


In [9]:
#this is for  regressors predicting wins - losses, can use this to turn output into win prediction 

def make_win(x):
    if x <= 0:
        return 0
    if x >0:
        return 1

v_make_win = np.vectorize(make_win)

#useage: v_make_win(y_pred)

In [10]:
def regr_model_results(model, model_name, X, dates, step, window_size, prediction_size, drop_first_k_days = 0): #X = data 
    results_dic['model_name'] = []
    results_dic['date'] = []
    results_dic['mae'] = []
    results_dic['mse'] = []
    results_dic['r2'] = []   
    
    
    #drop first k days from dates and X
    dates = dates[drop_first_k_days :]
    X = X.loc[X['full_date'].isin(dates), :].copy()  

    for i in range(step, len(dates), step): ##eg step =10, so 17 rounds
        
        model.fit(X.loc[X['full_date'].isin(dates[max(i-window_size ,0):i]), :],y.loc[y['full_date'].isin(dates[max(i-window_size,0):i]),'goal_difference' ])
        y_pred = model.predict(X.loc[X['full_date'].isin(dates[i:i+prediction_size]), :])
        y_test = y.loc[y['full_date'].isin(dates[i:i+prediction_size]),'goal_difference' ]
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results_dic['model_name'].append(model_name)
        results_dic['date'].append(dates[i])
    
        results_dic['mae'].append(mae)
        results_dic['mse'].append(mse)
        results_dic['r2'].append(r2)
        
    return results_dic #!
        
        

In [11]:
def class_model_results(model, model_name, X, dates, step, window_size, prediction_size, drop_first_k_days = 0): #X = data 
    results_dic ={}
    results_dic['model_name'] = []
    results_dic['date'] = []
    results_dic['accuracy'] = []
    results_dic['f1_score'] = []

    #results_dic['precision'] = []
  #  results_dic['recall'] = []
    
    #drop first k days from dates and X
    dates = dates[drop_first_k_days :]
    X = X.loc[X['full_date'].isin(dates), :].copy()  

    for i in range(step, len(dates), step): ##eg step =10, so 17 rounds
        model.fit(X.loc[X['full_date'].isin(dates[max(i-window_size ,0):i]), :],y.loc[y['full_date'].isin(dates[max(i-window_size,0):i]),'won' ])
        y_pred = model.predict(X.loc[X['full_date'].isin(dates[i:i+prediction_size]), :])
        y_test = y.loc[y['full_date'].isin(dates[i:i+prediction_size]),'won' ]
    
        accuracy = accuracy_score(y_test, y_pred)
        #recision = precision_score(y_test, y_pred, zero_division = 0)
        #recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred) #, average = None)
            
        results_dic['model_name'].append(model_name)  #append same model name every iter so same length as others
        results_dic['date'].append(dates[i])
                          
        results_dic['accuracy'].append(accuracy)
        results_dic['f1_score'].append(f1)
        #results_dic['precision'].append(precision)
        #results_dic['recall'].append(recall)
    results_dic['model_name'].append('model_name'+'_avg')  #append same model name every iter so same length as others
    results_dic['date'].append('average')
    results_dic['accuracy'].append(round(np.mean(np.array(results_dic['accuracy'])), 2) ) 
    results_dic['f1_score'].append(round(np.mean(np.array(results_dic['f1_score'])), 2) ) 
    return results_dic #!
        
        

1. two versions of dummies for teams - teams plus H/A and teamH - teamA 
2. play with add/remove when in season (hypothesis: offense early defense favored late) ---> 5 vars for where in year ... 

In [13]:
"dajfdajjj"[:-3]

'dajfda'

In [29]:
def make_diff(statFor, statAway):
    return statFor - statAway

v_make_diff = np.vectorize(make_diff)

def make_per(statFor, statAway):  #example FOWFor/(FOWFor + FOWAgainst)  or ShAt
    try: 
        return statFor/(statFor+statAway)
    except: 
        return 0


v_make_per = np.vectorize(make_per)
 
    
def make_ratio(stat1, stat2):  #example goalsFor/shotsFor = sh%
    try:
        return stat1/stat2
    except:
        return 0
    
v_make_ratio = np.vectorize(make_ratio)

##have to adjust later depending what is convenient to use for stat name ... 

#not using k_days_back .. will use a unioversal fn to just avg past games...
#makes more sense ,,, if they get a 1000 shots against one game, don't want that to leak into 
#other games .. 



def get_per(X, statFor): 
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = statFor[:-3]  #remove last 3
    #statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    X[statFor+'%' ] = v_make_per(X.loc[:,statFor],X.loc[:,statAgainst]) 
    #return v_make_per(X.loc[:,statFor],X.loc[:,statAgainst])


def get_ratio(X, stat1, stat2, new_stat_name):  #stat1/stat2  
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    
    X[new_stat_name] = v_make_ratio(X.loc[:,stat1],X.loc[:,stat2]) 
    #return v_make_ratio(X.loc[:,stat1],X.loc[:,stat2])

def get_diff(X, statFor): 
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = statFor[:-3]  #remove last 3
    #statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    X[stat_name+'Diff']= v_make_diff(X.loc[:,statFor],X.loc[:, statAgainst])
    #return v_make_diff(X.loc[:,statFor],X.loc[:, statAgainst])
    
    

In [15]:
[1,3,4, 7, 8][-3:]

[4, 7, 8]

In [16]:
k = 10

X_16 = data.loc[data['season'] == 20162017, :].copy()
all_dates = sorted(list(X_16['full_date']))

date1 = 20161022
dates_bef_date1 = [date for date in all_dates if date < date1]
dates_bef_date1 = dates_bef_date1[-k :]

stat_name = 'goalsFor'
sk = str(k)
X_16[stat_name+sk+'_avg'] = np.NaN 

X_bef_date1 = X_16.loc[X_16['full_date'].isin(dates_bef_date1), :]
teams_date1 = sorted(list(X_bef_date1['nhl_name']))
nhl_name = 'ANA' 
X = X_16.copy()
k_avg = np.mean(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+sk+'_avg'] =k_avg
k_avg    

3.0

In [17]:
all_dates = sorted(set((X_16['full_date'])))

X_16.head()

,date,season,Open,full_date,nhl_name,game_id,team_id,HoA,won,settled_in,...,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,goalsFor10_avg
19714,1012,20162017,114,20161012,TOR,2016020001,10,away,0,OT,...,40.626,8.0,0.417,0.302,0.302,2.222,2.149,2.139,0,NaN
19715,1012,20162017,-134,20161012,OTT,2016020001,9,home,1,OT,...,59.050,4.0,0.567,0.221,0.221,2.546,2.711,2.689,0,NaN
19716,1012,20162017,108,20161012,STL,2016020002,19,away,1,REG,...,23.765,10.0,0.310,0.698,0.698,1.394,1.370,1.340,0,NaN
19717,1012,20162017,-128,20161012,CHI,2016020002,16,home,0,REG,...,45.176,13.0,0.569,2.247,2.247,3.791,3.835,3.782,0,NaN
19718,1012,20162017,108,20161012,CGY,2016020003,20,away,0,REG,...,40.285,6.0,0.469,0.514,0.514,3.478,3.496,3.478,0,NaN


In [170]:
##!! you need to loop over nhl_name at the beginning so your dates are associated to that fixed team


#num of games back max(xurrent - k, 0) = 0 for 1000
     #here X_N = data.iloc[0:date-1, :] all games from all days before present day.  
    #here X = whole season or whatever chumk you want
def get_k_game_avg(X, stat_name,  k_days_back= 10**6 ): #k_days_back #season is random-default
   # for season in season_list:
    #X = data.loc[data['season'] == season, :].copy()      
    #X = data.copy()
    
    if k_days_back < 10**6:
        k = str(k_days_back)
    else: 
        k = "_cumul"
        
    all_dates = sorted(set(X['full_date']))
    X[stat_name+k+'_avg'] = np.NaN  #set up column
    
    #take care of first day of season ... might want to leave it NaN ... nahhh
    
    
    #date0 = all_dates[0]
    #X.loc[(X['full_date'] == date0), stat_name+k+'_avg'] =  X.loc[(X['full_date'] == date0), stat_name]
        
    for date1 in all_dates[1:]: #all but first date so don't get emopty           
        dates_bef_date1 = [date for date in all_dates if date < date1]
        dates_bef_date1 = dates_bef_date1[-k_days_back :] # this grabs all or the last k days of list
        
        X_bef_date1 = X.loc[X['full_date'].isin(dates_bef_date1), :]           
        teams_date1 = sorted(set(X_bef_date1['nhl_name']))
            
            #first take care of all teams not present in the previous days (only a few)
        for nhl_name in set(X['nhl_name']).difference(set(X_bef_date1['nhl_name'])):
            k_avg = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_avg'] =k_avg
        
        
        #this is the main step
        for nhl_name in teams_date1:
            k_avg = np.mean(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_avg'] =k_avg
   # return X

In [343]:
##!! you need to loop over nhl_name at the beginning so your dates are associated to that fixed team


def get_k_game_sum(X, stat_name, k_days_back= 10**6 ): #k_days_back #season is random-default
   # for season in season_list:
    #X = data.loc[data['season'] == season, :].copy()      
    #X = data.copy()
    if k_days_back < 10**6:
        k = '_'+str(k_days_back)+'_day'
    else: 
        k = "_cumul"
    all_dates = sorted(set(X['full_date']))
    X[stat_name+k+'_sum'] = np.NaN  #set up column
    
    #take care of first day of season ... might want to leave it NaN ... nahhh
    #date0 = all_dates[0]
    #X.loc[(X['full_date'] == date0), stat_name+k+'_sum'] =  X.loc[(X['full_date'] == date0), stat_name]
        
    for date1 in all_dates[1:]: #all but first date so don't get emopty           
        dates_bef_date1 = [date for date in all_dates if date < date1]
        dates_bef_date1 = dates_bef_date1[-k_days_back :] # this grabs all or the last k days of list
        
        X_bef_date1 = X.loc[X['full_date'].isin(dates_bef_date1), :]           
        teams_date1 = sorted(set(X_bef_date1['nhl_name']))
            
            #first take care of all teams not present in the previous days (only a few)
        for nhl_name in set(X['nhl_name']).difference(set(X_bef_date1['nhl_name'])):
            k_sum = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_sum'] =k_sum
        
        
        #this is the main step
        for nhl_name in teams_date1:
            k_sum = np.sum(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_sum'] =k_sum
   # return X

In [19]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

In [20]:
feat_Pis_plus = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

In [21]:
#features I need to do this ...
feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
#'win_streak_grouped_10', 'conference_standing_grouped_10',
#'PDO'  

#targets 
 #could later try to train classifier to predict this ... tough tho --> reg or no
    

extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]



In [22]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
###set Leung aside for now ... 
    
feat_Leung = ['ID', 'Date', 'HomeTeam', 'AwayTeam', 'GDiff','GF%', 'CF%','CSh%',
'CSv%', 'FF%','FSh%','FSv%','PDO','PENDiff','ShF%','SDiff','Sh%', 'Sv%','FOW%','W%','FavoritesW%', 'Result']


feat_for_Leung  = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]
    
    
feat_plus = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%',  ]


In [24]:
data_Pis = data.loc[:, feat_for_Pis].copy()

In [25]:
data_Pis.head()

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,shotsOnGoalAgainst,shotsOnGoalFor,savedShotsOnGoalAgainst,savedShotsOnGoalFor,fenwickPercentage,won,settled_in
0,20081004,20082009,2008020001,NYR,away,T.B,1.0,2.0,1.0,7.0,21.0,41.0,20.0,39.0,0.6310,1,REG
1,20081004,20082009,2008020001,TBL,home,NYR,2.0,1.0,0.0,4.0,41.0,21.0,39.0,20.0,0.3690,0,REG
2,20081004,20082009,2008020002,PIT,away,OTT,3.0,4.0,0.0,7.0,35.0,30.0,32.0,26.0,0.4778,1,OT
3,20081004,20082009,2008020002,OTT,home,PIT,4.0,3.0,1.0,6.0,30.0,35.0,26.0,32.0,0.5222,0,OT
4,20081005,20082009,2008020003,TBL,away,NYR,2.0,1.0,0.0,5.0,39.0,19.0,37.0,18.0,0.3810,0,REG


In [173]:
#season = 20122013  #select this shortened season because same as Pisch

X_12 = data.loc[data['season'] ==20122013, :].copy()


In [109]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goalDiff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 
            'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 


#X_12.head()

In [174]:

feat_for_Pis_small = [ 'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage',] 

get_diff(X_12, 'goalsFor')



In [175]:

get_per(X_12, 'goalsFor')

In [176]:
get_ratio(X_12,  'powerPlayGoals','powerPlayOpportunities', 'pp%')

In [113]:


def one_plus(x):
    return 1+x

v_one_plus = np.vectorize(one_plus)

def minus_one(x):
    return x-1

v_minus_one = np.vectorize(minus_one)

def one_minus(x):
    return 1-x

v_one_minus = np.vectorize(one_minus)

In [177]:
#thing1.apply(one_minus)

X_12['pk%'] = 0


In [178]:
X_12.sort_values(by = ['full_date', 'game_id', 'HoA'], inplace = True)


In [179]:
X_12.reset_index(drop = True, inplace = True)
#X_12['season'].value_counts()

In [181]:
#thing1 = X_12.loc[X_12['HoA'] == 'away' ,['pp%']].copy().apply(one_minus).copy()

X_12.loc[X_12['HoA'] == 'home', 'pk%']  = v_one_minus(X_12.loc[X_12['HoA'] == 'away' ,['pp%']].copy())


In [182]:
#thing2  = X_12.loc[X_12['HoA'] == 'home' ,['pp%']].copy().apply(one_minus).copy()

X_12.loc[X_12['HoA'] == 'away',['pk%']] =  v_one_minus(X_12.loc[X_12['HoA'] == 'home' ,['pp%']].copy())



In [81]:
X_12.loc[:, ['pk%']].isnull().sum()
X_12.loc[:10, ['HoA', 'pp%', 'pk%']]

,HoA,pp%,pk%
0,away,0.666667,1.000000
1,home,0.000000,0.333333
2,away,0.500000,0.833333
3,home,0.166667,0.500000
4,away,0.200000,1.000000
5,home,0.000000,0.800000
6,away,0.000000,1.000000
7,home,0.000000,1.000000
8,away,0.400000,0.800000
9,home,0.200000,0.600000


In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  

In [183]:
get_ratio(X_12, 'goalsFor', 'shotsOnGoalFor', 'sh%')

In [184]:
get_ratio(X_12, 'goalsAgainst', 'shotsOnGoalAgainst', 'sv%')

In [185]:
X_12['sv%'] = one_minus(X_12['sv%'])

In [70]:
X_12.head(2)

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,...,settled_in,goalsDiff,goalsFor%,pp%,pk%,sh%,sv%,ones,ones3000_avg,ones3000_sum
0,20130119,20122013,2012020001,PIT,away,PHI,1.0,3.0,2.0,3.0,...,REG,2.0,0.75,0.666667,1.000000,0.111111,0.962963,1,1.0,NaN
1,20130119,20122013,2012020001,PHI,home,PIT,3.0,1.0,0.0,5.0,...,REG,-2.0,0.25,0.000000,0.333333,0.037037,0.888889,1,1.0,NaN


In [90]:
np.sum([1,2])

3

In [186]:
X_12['ones'] = 1

In [187]:
get_k_game_sum(X_12, 'won') 

In [188]:
get_k_game_sum(X_12, 'ones')

In [189]:
X_12.rename(columns = {'ones_cumul_sum': 'games_so_far'}, inplace = True)

In [126]:
X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'sh%', 'sv%', 'ones',
       'won_cumul_sum', 'games_so_far'],
      dtype='object')

In [130]:
X_12.loc[200:230, ['full_date','HoA','nhl_name', 'won_cumul_sum', 'games_so_far', 'win%']]

,full_date,HoA,nhl_name,won_cumul_sum,games_so_far,win%
200,20130201,away,PHI,2.0,7.0,0.285714
201,20130201,home,WSH,1.0,7.0,0.142857
202,20130201,away,OTT,5.0,7.0,0.714286
203,20130201,home,CAR,2.0,5.0,0.400000
204,20130201,away,WPG,3.0,7.0,0.428571
205,20130201,home,TBL,5.0,6.0,0.833333
206,20130201,away,STL,6.0,7.0,0.857143
207,20130201,home,DET,3.0,6.0,0.500000
208,20130201,away,ARI,2.0,7.0,0.285714
209,20130201,home,DAL,2.0,7.0,0.285714


In [190]:
X_12['PDO'] = X_12['sh%'] + X_12['sv%']

In [102]:
##I am stuck on this win% thing ... let me try a model ... for now

X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'ones', 'onescumul_avg',
       'games_so_far', 'sh%', 'sv%', 'PDO'],
      dtype='object')

In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 


In [191]:
get_ratio(X_12, 'won_cumul_sum', 'games_so_far', 'win%')

In [193]:
get_k_game_sum(X_12, 'won', k_days_back=10)

In [194]:
get_k_game_sum(X_12, 'ones', k_days_back=10)

In [136]:
X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'sh%', 'sv%', 'ones',
       'won_cumul_sum', 'games_so_far', 'win%', 'won_10day_sum',
       'ones_10day_sum'],
      dtype='object')

In [196]:
get_ratio(X_12, 'won_10_day_sum','ones_10_day_sum','last_10_games_win%')

In [139]:
X_12.head()

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,...,pk%,sh%,sv%,ones,won_cumul_sum,games_so_far,win%,won_10day_sum,ones_10day_sum,last_10_games_win%
0,20130119,20122013,2012020001,PIT,away,PHI,1.0,3.0,2.0,3.0,...,1.000000,0.111111,0.962963,1,1.0,1.0,1.0,1.0,1.0,1.0
1,20130119,20122013,2012020001,PHI,home,PIT,3.0,1.0,0.0,5.0,...,0.333333,0.037037,0.888889,1,0.0,1.0,0.0,0.0,1.0,0.0
2,20130119,20122013,2012020002,OTT,away,WPG,1.0,4.0,1.0,2.0,...,0.833333,0.108108,0.964286,1,1.0,1.0,1.0,1.0,1.0,1.0
3,20130119,20122013,2012020002,WPG,home,OTT,4.0,1.0,1.0,6.0,...,0.500000,0.035714,0.891892,1,0.0,1.0,0.0,0.0,1.0,0.0
4,20130119,20122013,2012020003,CHI,away,L.A,2.0,5.0,1.0,5.0,...,1.000000,0.227273,0.904762,1,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 


In [197]:
feat_Pis_to_sum  = ['goalsAgainst', 'goalsFor', 'goalsDiff']
feat_Pis_to_avg = ['goalsFor%',  'pp%', 'pk%',  'sh%', 'sv%', 'fenwickPercentage', 'PDO',    ]  

##NOTE! This is a bit inaccurate to average these ... one should find cumul_sums then do the % calculations above for accurate %
##let's see if they are that different ... 

#'last_10_games_win%',  'win%',     omtted because already a 10 gm avg, 

for feat_to_sum in feat_Pis_to_sum:
    get_k_game_sum(X_12, feat_to_sum)
    
    

In [198]:
for feat_to_avg in feat_Pis_to_avg:
    get_k_game_avg(X_12, feat_to_avg)

In [200]:
X_12.head(30)

,date,season,Open,full_date,nhl_name,game_id,team_id,HoA,won,settled_in,...,goalsAgainst_cumul_sum,goalsFor_cumul_sum,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg
0,119,20122013,-115,20130119,PIT,2012020001,5,away,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119,20122013,-105,20130119,PHI,2012020001,4,home,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119,20122013,100,20130119,OTT,2012020002,9,away,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119,20122013,-120,20130119,WPG,2012020002,52,home,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,119,20122013,110,20130119,CHI,2012020003,16,away,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,119,20122013,-130,20130119,LAK,2012020003,26,home,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,119,20122013,100,20130119,NYR,2012020004,3,away,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,119,20122013,-120,20130119,BOS,2012020004,6,home,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,119,20122013,115,20130119,TOR,2012020005,10,away,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,119,20122013,-135,20130119,MTL,2012020005,8,home,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'sh%', 'sv%', 'ones',
       'won_cumul_sum', 'games_so_far', 'win%', 'won_10day_sum',
       'ones_10day_sum', 'last_10_games_win%', 'PDO', 'goalsAgainst_cumul_sum',
       'goalsFor_cumul_sum', 'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 'sh%_cumul_avg', 'sv%_cumul_avg',
       'fenwickPercentage_cumul_avg', 'PDO_cumul_avg'],
      dtype='object')

In [317]:
X_12.iloc[20:24, :]

,date,season,Open,full_date,nhl_name,game_id,team_id,HoA,won,settled_in,...,goalsAgainst_cumul_sum,goalsFor_cumul_sum,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg
20,119,20122013,105,20130119,ARI,2012020011,27,away,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,119,20122013,-125,20130119,DAL,2012020011,25,home,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,119,20122013,110,20130119,COL,2012020012,21,away,0,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,119,20122013,-130,20130119,MIN,2012020012,30,home,1,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#feat_Pis_to_sum  = ['goalsAgainst', 'goalsFor', 'goalsDiff']
#feat_Pis_to_avg = ['goalsFor%',  'pp%', 'pk%',  'sh%', 'sv%', 'fenwickPercentage', 'PDO',    ]  


In [328]:
#bad_game_ids with 0 0 score (in df_game_team_stats) ,df_game is probably ok 
bade_game_ids = [2008020057, 2008020071, 2008020306, 2008020623, 2008021108,
            2008021196, 2009020072, 2009020253, 2009020682, 2009020831,
            2009021118, 2009021209, 2010020382, 2010020761, 2010020878,
            2010021111, 2011020749, 2011020787, 2011021016, 2011021052,
            2011021108, 2012020159, 2012020412, 2012020487, 2013020126,
            2013021136, 2013021223, 2014020055, 2014020158, 2014020313,
            2014020456, 2014021008, 2014021210, 2016020785, 2017020561,
            2017020965, 2018020783, 2019020127, 2019021041]

In [329]:
##check stats calculations first few games ... 
##part 1 ... goals for and against stats ... 
##they are all OT 1-0 wins ... so you can impute this using the "won" column

X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won', 'settled_in','games_so_far', 'goalsAgainst',  'goalsFor', 'goalsFor%', 'goalsDiff','goalsDiff_cumul_sum','goalsAgainst_cumul_sum',   'goalsFor_cumul_sum',    'goalsFor%_cumul_avg', 
   ]].iloc[:15, :]

,game_id,full_date,nhl_name,HoA,won,settled_in,games_so_far,goalsAgainst,goalsFor,goalsFor%,goalsDiff,goalsDiff_cumul_sum,goalsAgainst_cumul_sum,goalsFor_cumul_sum,goalsFor%_cumul_avg
30,2012020016,20130120,SJS,away,1,REG,1.0,1.0,4.0,0.800000,3.0,3.0,1.0,4.0,0.800000
68,2012020035,20130122,SJS,away,1,REG,1.0,3.0,6.0,0.666667,3.0,3.0,1.0,4.0,0.800000
97,2012020049,20130124,SJS,home,1,REG,2.0,3.0,5.0,0.625000,2.0,6.0,4.0,10.0,0.733333
113,2012020057,20130126,SJS,home,1,REG,3.0,0.0,4.0,1.000000,4.0,8.0,7.0,15.0,0.697222
143,2012020072,20130127,SJS,home,1,REG,4.0,1.0,4.0,0.800000,3.0,12.0,7.0,19.0,0.772917
173,2012020087,20130129,SJS,home,1,OT,5.0,2.0,2.0,0.500000,0.0,15.0,8.0,23.0,0.778333
199,2012020100,20130131,SJS,home,1,OT,6.0,2.0,2.0,0.500000,0.0,15.0,10.0,25.0,0.731944
235,2012020118,20130202,SJS,home,0,OT,7.0,1.0,1.0,0.500000,0.0,15.0,12.0,27.0,0.698810
252,2012020127,20130204,SJS,away,0,REG,8.0,2.0,1.0,0.333333,-1.0,15.0,13.0,28.0,0.673958
273,2012020137,20130205,SJS,home,0,REG,9.0,5.0,3.0,0.375000,-2.0,14.0,15.0,29.0,0.636111


In [ ]:
'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage',    'sh%_cumul_avg', 'sv%_cumul_avg', 'PDO',  'PDO_cumul_avg'
       'fenwickPercentage_cumul_avg',
     'sh%', 'sv%', 
      'powerPlayGoals', 'powerPlayOpportunities',
            'pp%', 'pk%', 'pp%_cumul_avg', 'pk%_cumul_avg',
       

In [ ]:
##check stats calculations first few games ... 
##part 4 ...shots, fenwick 
X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won','goalsAgainst',  'goalsFor', 
       'powerPlayGoals', 'powerPlayOpportunities',
            'pp%', 'pk%', 'pp%_cumul_avg', 'pk%_cumul_avg', ]].iloc[:20, :]

In [339]:
##check stats calculations first few games ... 
##part 3 ... pp, 
X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won','goalsAgainst',  'goalsFor', 
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst', 
       'savedShotsOnGoalFor', 'sh%', 'sv%','PDO', 'PDO_cumul_avg','sh%_cumul_avg', 'sv%_cumul_avg', 'fenwickPercentage',     
       'fenwickPercentage_cumul_avg',
      ]].iloc[-8:, :]

,game_id,full_date,nhl_name,HoA,won,goalsAgainst,goalsFor,shotsOnGoalAgainst,shotsOnGoalFor,savedShotsOnGoalAgainst,savedShotsOnGoalFor,sh%,sv%,PDO,PDO_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage,fenwickPercentage_cumul_avg
1232,2012020618,20130413,SJS,away,0,2.0,1.0,19.0,32.0,17.0,31.0,0.031250,0.894737,0.925987,0.993819,0.075528,0.918291,0.6795,0.522670
1256,2012020631,20130415,SJS,away,1,0.0,4.0,35.0,41.0,35.0,37.0,0.097561,1.000000,1.097561,0.992165,0.074448,0.917717,0.4907,0.526495
1273,2012020639,20130416,SJS,home,1,2.0,2.0,43.0,35.0,41.0,33.0,0.057143,0.953488,1.010631,0.994674,0.074998,0.919676,0.4700,0.525643
1301,2012020653,20130418,SJS,home,1,1.0,6.0,28.0,29.0,27.0,23.0,0.206897,0.964286,1.171182,0.995045,0.074583,0.920462,0.5000,0.524349
1341,2012020674,20130421,SJS,home,0,4.0,3.0,22.0,35.0,18.0,32.0,0.085714,0.818182,0.903896,0.999049,0.077590,0.921458,0.6000,0.523795
1373,2012020689,20130423,SJS,home,1,2.0,3.0,26.0,37.0,24.0,34.0,0.081081,0.923077,1.004158,0.996934,0.077771,0.919163,0.5699,0.525489
1380,2012020693,20130424,SJS,away,0,2.0,1.0,32.0,34.0,30.0,33.0,0.029412,0.937500,0.966912,0.997091,0.077843,0.919248,0.5111,0.526454
1436,2012020720,20130427,SJS,away,0,3.0,2.0,27.0,25.0,24.0,23.0,0.080000,0.888889,0.968889,0.996449,0.076812,0.919637,0.4125,0.526128


In [325]:
##check stats calculations first few games ... 
##part 2 ... wins, win% ...
X_12.loc[X_12['nhl_name'].isin(['SJS']), ['game_id','full_date', 'nhl_name', 'HoA', 'won','goalsAgainst',  'goalsFor', 
       'won_cumul_sum', 'games_so_far', 'win%', 'ones','won_10_day_sum',
       'ones_10_day_sum', 'last_10_games_win%', ]].iloc[:20, :]

,game_id,full_date,nhl_name,HoA,won,goalsAgainst,goalsFor,won_cumul_sum,games_so_far,win%,ones,won_10_day_sum,ones_10_day_sum,last_10_games_win%
30,2012020016,20130120,SJS,away,1,1.0,4.0,1.0,1.0,1.000000,1,1.0,1.0,1.000000
68,2012020035,20130122,SJS,away,1,3.0,6.0,1.0,1.0,1.000000,1,1.0,1.0,1.000000
97,2012020049,20130124,SJS,home,1,3.0,5.0,2.0,2.0,1.000000,1,2.0,2.0,1.000000
113,2012020057,20130126,SJS,home,1,0.0,4.0,3.0,3.0,1.000000,1,3.0,3.0,1.000000
143,2012020072,20130127,SJS,home,1,1.0,4.0,4.0,4.0,1.000000,1,4.0,4.0,1.000000
173,2012020087,20130129,SJS,home,1,2.0,2.0,5.0,5.0,1.000000,1,5.0,5.0,1.000000
199,2012020100,20130131,SJS,home,1,2.0,2.0,6.0,6.0,1.000000,1,5.0,5.0,1.000000
235,2012020118,20130202,SJS,home,0,1.0,1.0,7.0,7.0,1.000000,1,5.0,5.0,1.000000
252,2012020127,20130204,SJS,away,0,2.0,1.0,7.0,8.0,0.875000,1,4.0,5.0,0.800000
273,2012020137,20130205,SJS,home,0,5.0,3.0,7.0,9.0,0.777778,1,4.0,6.0,0.666667


In [242]:
date_first = X_12['full_date'][0].copy()
#date_first
data_Pis_pre = X_12.loc[X_12['full_date'] > date_first, ['HoA','goalsAgainst_cumul_sum',
       'goalsFor_cumul_sum', 'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 'sh%_cumul_avg', 'sv%_cumul_avg',
       'fenwickPercentage_cumul_avg', 'PDO_cumul_avg','last_10_games_win%',   'win%',  ]].copy()


In [243]:
data_Pis_H = data_Pis_pre.loc[data_Pis_pre['HoA'] =='home',:].copy()
data_Pis_A = data_Pis_pre.loc[data_Pis_pre['HoA'] =='away',:].copy()


In [246]:
data_Pis_A.reset_index(drop = True, inplace = True)
data_Pis_H.reset_index(drop = True, inplace = True)


In [253]:
data_Pis = data_Pis_H.iloc[:, 1:].copy().sub(data_Pis_A.iloc[:, 1:].copy()).copy()

In [254]:
data_Pis

,goalsAgainst_cumul_sum,goalsFor_cumul_sum,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg,last_10_games_win%,win%
0,-1.0,4.0,5.0,0.464286,0.500000,0.416667,0.084914,0.042146,0.067200,0.127060,1.0,1.000000
1,2.0,-2.0,-4.0,-0.500000,-0.666667,0.000000,-0.063492,-0.051198,-0.075100,-0.114690,-1.0,-1.000000
2,3.0,-3.0,-6.0,-0.600000,0.000000,0.000000,-0.106681,-0.106681,0.108400,-0.213362,-1.0,-1.000000
3,-1.0,0.0,1.0,0.095238,-0.214286,0.400000,-0.039560,0.000926,0.062100,-0.038635,0.0,0.000000
4,5.0,1.0,-4.0,-0.200000,0.250000,-1.000000,0.040948,-0.200265,0.092100,-0.159317,-1.0,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
702,28.0,9.0,-19.0,-0.035630,-0.007143,-0.042334,0.015628,-0.011179,-0.064266,0.004450,-0.4,-0.021277
703,21.0,-4.0,-25.0,-0.094540,0.043558,-0.022948,-0.000104,-0.004858,-0.065536,-0.004962,-0.1,-0.170213
704,-12.0,15.0,27.0,0.091067,0.065907,-0.002668,0.023944,0.002161,-0.018613,0.026105,0.2,0.212766
705,3.0,14.0,11.0,0.029841,-0.001672,-0.038180,0.021633,-0.019687,0.023549,0.001946,0.0,0.021277


In [257]:
dummies_pm1_Pis, y_Pis  = make_HA_diff(data, season = 20122013)

In [267]:
dummies_pm1_Pis_final = dummies_pm1_Pis.loc[dummies_pm1_Pis['full_date'] > date_first, :].iloc[:, :30].reset_index(drop = True).copy()

In [252]:
dummies_pm1_Pis = dummies_pm1_Pis.iloc[:, :30].copy() 
dummies_pm1_Pis.iloc[26:, :]

,ANA,ARI,BOS,BUF,CAR,CBJ,CGY,CHI,COL,DAL,...,OTT,PHI,PIT,SJS,STL,TBL,TOR,VAN,WPG,WSH
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-1
27,0,0,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
30,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0,0,0,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
716,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
717,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
def make_HA_diff(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    df_model = df_home.sub(df_visitor) 
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)


In [268]:
data_dummies_Pis = pd.concat([dummies_pm1_Pis_final, data_Pis], axis =1)
#bit of a hack soln ... 26 is from the first day games dropped in data_Pis

In [345]:
data_dummies_Pis.to_csv('data_dummies_Pis_v0.csv')

In [344]:
data_dummies_Pis.iloc[100:120,:]

,ANA,ARI,BOS,BUF,CAR,CBJ,CGY,CHI,COL,DAL,...,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg,last_10_games_win%,win%
100,0,0,0,0,1,0,0,0,0,0,...,-11.0,-0.105357,0.044048,0.046471,-0.027065,-0.009299,-0.031307,-0.036365,-0.583333,-0.321429
101,0,-1,0,0,0,0,0,0,0,1,...,5.0,0.137946,0.106250,-0.030060,0.015631,-0.029763,0.070912,-0.014132,0.133333,-0.125000
102,0,0,0,0,0,0,-1,1,0,0,...,-15.0,-0.187778,0.086429,-0.254167,-0.019152,-0.088077,0.036440,-0.107229,-0.266667,-0.550000
103,-1,0,0,0,0,0,0,0,0,0,...,6.0,0.078307,0.180556,-0.128704,0.054635,-0.009929,-0.044533,0.044706,0.000000,0.333333
104,0,0,0,0,0,0,0,0,0,0,...,20.0,0.305952,0.043197,0.085714,0.059054,0.028441,0.057543,0.087495,0.750000,0.714286
105,0,0,0,0,0,0,0,0,0,0,...,-13.0,-0.162798,-0.017708,0.021429,-0.010650,-0.020188,-0.067412,-0.030838,-0.266667,-0.375000
106,0,0,0,0,0,0,0,0,0,0,...,-3.0,-0.024702,0.021131,-0.093750,0.028874,-0.037118,-0.003102,-0.008244,0.300000,0.089286
107,0,0,0,-1,0,0,0,0,0,0,...,6.0,0.109265,0.071131,0.056888,0.016477,0.015461,0.014820,0.031938,-0.083333,0.089286
108,0,0,0,0,0,0,0,0,0,0,...,5.0,-0.018841,0.340476,0.153571,0.039626,-0.018266,-0.025871,0.021360,0.400000,0.142857
109,0,0,0,0,1,0,0,0,0,0,...,3.0,-0.062879,0.020833,0.066071,0.027962,0.021689,-0.064782,0.049651,0.000000,0.071429
